# ocena modelu

In [ ]:
#embbeding_network = wytrenowany model

In [ ]:
import numpy as np

#funkcje operujące na wektorach zwracające spośród listy wektorów
#wektor najbliższy do wektora podanego w parametrach wywołania funkcji

def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

def top1_accuracy(vector_anchor, list_of_vectors):
    closest_vec = None
    min_distance = float('inf')
    for vector in list_of_vectors:
        distance = euclidean_distance(vector_anchor, vector)
        if distance < min_distance:
            min_distance = distance
            closest_vec = vector
    return closest_vec

def top3_accuracy(vector_anchor, list_of_vectors):
    # Oblicz odległości od wektora kotwicy dla wszystkich wektorów w liście
    distances = [(vector, euclidean_distance(vector_anchor, vector)) for vector in list_of_vectors]
    
    # Posortuj według odległości rosnąco
    distances.sort(key=lambda x: x[1])
    
    # Zwroc pierwsze trzy elementy (najbliższe wektory)
    closest_3_vecs = [item[0] for item in distances[:3]]
    return closest_3_vecs

In [ ]:
# Uzyskaj model do generowania embeddingów
embedding_model = embedding_network

In [ ]:
# Ścieżka do głównego folderu zbioru testowego
base_test_folder = "sciezka_do_bazy_danych/fruits-360_dataset/fruits-360/Test/"

# Lista ścieżek do pierwszego zdjęcia w każdym z podfolderów
first_image_paths = []

# Przeglądanie każdego folderu w zbiorze testowym
for root, dirs, files in os.walk(base_test_folder):
    # Sprawdź, czy bieżący folder ma jakieś pliki
    if files:
        # Sortowanie plików w celu uzyskania pierwszego według nazwy
        files_sorted = sorted(files)
        # Dodaj pełną ścieżkę do pierwszego pliku
        first_image_path = os.path.join(root, files_sorted[0])
        first_image_paths.append(first_image_path)

# Wyświetlenie listy ścieżek
for path in first_image_paths:
    print(path)

In [ ]:
first_images = [mapF.decode_and_resize(image_path) for image_path in first_image_paths]

# Oblicz embedingi dla referencyjnych obrazów
first_embeddings = embedding_model.predict(np.array(first_images))

In [ ]:
# Przygotuj obraz zapytania
query_image_path = "sciezka_do_zdjecia_zapytania_(query)"
query_image = mapF.decode_and_resize(query_image_path)

# Oblicz embedding dla obrazu zapytania
query_embedding = embedding_model.predict(np.array([query_image]))[0]

In [ ]:
# Znajdź najbliższy obraz (Top-1)
closest_vector = top1_accuracy(query_embedding, first_embeddings)

# Ustal indeks najbliższego wektora
closest_image_index = np.argmin([euclidean_distance(closest_vector, vec) for vec in first_embeddings])

# Wydrukuj ścieżkę do najbliższego obrazu
print("Najbliższy obraz:", first_image_paths[closest_image_index])

# Uzyskaj trzy najbliższe wektory (Top-3)
closest_3_vectors = top3_accuracy(query_embedding, first_embeddings)

# Znajdź indeksy odpowiadające tym wektorom w liście referencyjnej
closest_3_indices = [np.argmin([euclidean_distance(vec, ref_vec) for ref_vec in first_embeddings]) for vec in closest_3_vectors]


# Wydrukuj ścieżki do trzech najbliższych obrazów
print("Trzy najbliższe obrazy:", [first_image_paths[i] for i in closest_3_indices])

In [ ]:
# Prawidłowe indeksy dla trzech najbliższych wektorów
closest_3_indices = [
    np.argmin([euclidean_distance(vec, ref_vec) for ref_vec in first_embeddings])
    for vec in closest_3_vectors
]

# Prawidłowy indeks dla top-1
closest_1_index = np.argmin([euclidean_distance(closest_vector, vec) for vec in first_embeddings])

# Wyświetl top-1 obraz
plt.figure(figsize=(2, 2))  # Rozmiar figury 2x2 cale
top1_image = mpimg.imread(first_image_paths[closest_1_index])

plt.title("top1")  # Użycie numeracji sekwencyjnej
plt.imshow(top1_image)
plt.axis("off")  # Wyłączenie osi dla lepszego widoku
plt.show()

# Załaduj i wyświetl obrazy z listy 'closest_3_indices'
plt.figure(figsize=(6, 2))  # Większy rozmiar dla 3 obrazów
for idx, index in enumerate(closest_3_indices):
    img = mpimg.imread(first_image_paths[index])
    plt.subplot(1, 3, idx + 1)  # 1 rząd, 3 kolumny, indeksy idą od 1 do 3
    plt.title(f"top {idx + 1}")  # Użycie numeracji sekwencyjnej
    plt.imshow(img)
    plt.axis("off")  # Wyłączenie osi dla lepszego widoku

# Wyświetlenie wszystkich obrazów obok siebie w jednym wierszu
plt.tight_layout()  # Ustawienie automatyczne dla lepszego rozmieszczenia
plt.show()  # Wyświetlenie wszystkich subplotów na raz